<a href="https://colab.research.google.com/github/gustavoSutil/materia-IA/blob/main/deep-learn/tf/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Estudante/Matrícula**: Gustavo Guilherme Sutil Alberton/2111100025

# Deep Learn in German Credit Numeric dataset
https://www.tensorflow.org/datasets/catalog/german_credit_numeric?hl=pt-br
https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data

Dependências

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Carregando o conjunto de dados
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.german_credit_numeric.load_data()

# Normalizando os dados (opcional, mas geralmente uma boa prática)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Construindo o modelo linear
model = models.Sequential()
model.add(layers.Dense(1, activation='sigmoid', input_shape=(x_train.shape[1],)))

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o modelo
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Avaliando o modelo
accuracy = model.evaluate(x_test, y_test)[1]
print(f'Acurácia do modelo linear: {accuracy}')


AttributeError: ignored